In [9]:
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os, shutil
import time
import scipy
from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras import models
from keras import layers
from sklearn.utils import class_weight

# Now lets try and to make the results better

In [10]:
# Create a data generator
datagen2 = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    rotation_range=20, # randomly rotate images in the range
    zoom_range=0.2, # randomly zoom image 
    width_shift_range=0.2, # randomly shift images horizontally
    height_shift_range=0.2, # randomly shift images vertically 
    horizontal_flip=True) # randomly flip images

In [11]:
# Use flow_from_directory to read images from folders
train_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/train', # point to the parent directory
        target_size=(299, 299), # resize images to 299x299
        batch_size=32,
        class_mode='binary') # binary for two classes

# Similarly create validation and test generators
val_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/val',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')

test_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/test',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')


Found 4098 images belonging to 2 classes.
Found 879 images belonging to 2 classes.
Found 879 images belonging to 2 classes.


In [12]:
model2 = models.Sequential()
model2.add(layers.Conv2D(32, (5, 5), activation='relu',
                         input_shape=(299 , 299,  3)))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(32, (4, 4), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(64, activation='relu')) # added another dense layer
model2.add(layers.Dense(1, activation='sigmoid'))


model2.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [13]:
from collections import Counter
import numpy as np

# Get class distribution
counter = Counter(train_generator2.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

print(counter)
print(max_val)
print(class_weights)

Counter({1: 2991, 0: 1107})
2991.0
{0: 2.7018970189701896, 1: 1.0}


In [14]:
epochs = 30

steps_per_epoch2 = np.ceil(train_generator2.samples / train_generator2.batch_size)
validation_steps2 = np.ceil(val_generator2.samples / val_generator2.batch_size)

history = model2.fit(
    train_generator2,
    steps_per_epoch=int(steps_per_epoch2),
    epochs=epochs,
    validation_data=val_generator2,
    validation_steps=int(validation_steps2),
    class_weight=class_weights)



Epoch 1/30


2023-07-12 20:09:33.025420: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - ETA: 0s - loss: 0.7787 - acc: 0.7343

2023-07-12 20:10:41.255033: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - 84s 619ms/step - loss: 0.7787 - acc: 0.7343 - val_loss: 0.3848 - val_acc: 0.8578
Epoch 2/30
129/129 [==============================] - 79s 609ms/step - loss: 0.5548 - acc: 0.8346 - val_loss: 0.4033 - val_acc: 0.8601
Epoch 3/30
129/129 [==============================] - 84s 652ms/step - loss: 0.5190 - acc: 0.8453 - val_loss: 0.2747 - val_acc: 0.8828
Epoch 4/30
129/129 [==============================] - 84s 640ms/step - loss: 0.4594 - acc: 0.8660 - val_loss: 0.2711 - val_acc: 0.8931
Epoch 5/30
129/129 [==============================] - 84s 645ms/step - loss: 0.4583 - acc: 0.8646 - val_loss: 0.3153 - val_acc: 0.8783
Epoch 6/30
129/129 [==============================] - 87s 655ms/step - loss: 0.4365 - acc: 0.8690 - val_loss: 0.2730 - val_acc: 0.8862
Epoch 7/30
129/129 [==============================] - 82s 635ms/step - loss: 0.4954 - acc: 0.8555 - val_loss: 0.2527 - val_acc: 0.8942
Epoch 8/30
129/129 [==============================] - 86s 659ms/st

In [15]:
results_train = model2.evaluate(train_generator2)
results_test = model2.evaluate(val_generator2)
print('Training set performance:', results_train)
print('Validation set performance:', results_test)

28/28 [==============================] - 14s 508ms/step - loss: 0.2125 - acc: 0.9113
Training set performance: [0.21117207407951355, 0.9170327186584473]
Validation set performance: [0.2125244140625, 0.9112628698348999]


## worse than last few models